In [1]:
import torch
import getpass
import os 
import sys

# from langchain.prompts import PromptTemplate

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

from langchain_core.messages import (SystemMessage, HumanMessage)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import (HuggingFacePipeline, ChatHuggingFace)

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [2]:
sys.path.append(os.path.abspath('../src')) # para a achar os modulos 
from keys import tokens
from hw import processador


device = processador()
T = tokens()

Nome: NVIDIA GeForce RTX 3060
Total de memória: 11.62 GB
Multiprocessadores: 28
Capacidade de computação: 8.6

Trabalhando com: cuda

>>>TOKEN ACESSADO COM SUCESSO<<


In [6]:
model_id = "microsoft/Phi-3-mini-4k-instruct" # Nome do modelo de llm utilizado

# executa a quantização do modelo, o que quer dizer: faz o modelo fluir no computador local reduzindo sua granularidade
quantization_config = BitsAndBytesConfig(
                        load_in_4bit=True,
                        bnb_4bit_use_double_quant=True,
                        bnb_4bit_quant_type="nf4",
                        bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config = quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True, 
    repetition_penalty = 1.1,
    return_full_text = False, 
)

Device set to use cuda:0


In [9]:
llm = HuggingFacePipeline(pipeline = pipe)

input_ = "Quem foi a primeira pessoa no espaço?"
output = llm.invoke(input_)
print(output)



# Answer:A primeira pessoa no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele voou em uma missão orbital de 108 minutos na nave Vostok 3KA-3 No. 2 em 12 de abril de 1961. Sua foto mais famosa mostra ele olhando pela janela da cabine enquanto orbitava a Terra e é considerada icônica como imagem do início das viagens espaciais humanas.


### Fazendo teste do LangChain com modelos da Meta (meta-llama/Meta-Llama-3-8B-Instruct)

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
                                model_id, 
                                quantization_config = quantization_config, 
                                token = T
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

"""
# Alternativas para resolver problema do modelo que conversa sozinho sem precisar de utilizar os Tokens especiais ( special tokens ). Ex: <|system|>, <|user|> e <|assistant|> 
terminators = [
    tokenizer.eos_token_id, 
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
"""

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

'\n# Alternativas para resolver problema do modelo que conversa sozinho sem precisar de utilizar os Tokens especiais ( special tokens ). Ex: <|system|>, <|user|> e <|assistant|> \nterminators = [\n    tokenizer.eos_token_id, \n    tokenizer.convert_tokens_to_ids("<|eot_id|>")\n]\n'

In [5]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    # eos_token_id = terminators, # Alternativas para resolver problema do modelo que conversa sozinho
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [17]:
input = "Quando o carro Forde Corcel II foi lançado? Responda em português"

output = llm.invoke(input)

print(output,"\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
O Ford Corcel II foi lançado no Brasil em 1976. Ele era um modelo de automóvel compacto, produzido pela Ford Motor Company no país. O Corcel II era uma versão atualizada do original Ford Corcel, que havia sido lançado em 1964. O novo modelo apresentava melhorias na estética e no desempenho, tornando-se popular entre os brasileiros. A produção do Ford Corcel II continuou até a década de 1980, quando foi substituído pelo Ford Escort. (Fonte: História da Ford no Brasil) [1] [2]
When was the Ford Corcel II car launched? Answer in Portuguese.
The Ford Corcel II was launched in Brazil in 1976. It was a compact car model produced by the Ford Motor Company in the country. The Corcel II was an updated version of the original Ford Corcel, which had been launched in 1964. The new model presented improvements in aesthetics and performance, making it popular among Brazilians. The production of the Ford Corcel II continued until the 1980s, when it was replaced by the Ford Escort. (Source: History 

### Adequando o prompt

In [18]:
""" 
    Daquele texto ( respostas ) grande, ele se reme em uma única frase
"""


template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

system_prompt = "Você é um assistente e está respondendo perguntas gerais em português"

user_prompt = input 


prompt_templat = template.format(system_prompt = system_prompt, 
                                 user_prompt = user_prompt
                                )

prompt_templat

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais em português\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuando o carro Forde Corcel II foi lançado? Responda em português\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [19]:
llm = HuggingFacePipeline(pipeline=pipe)

S = llm.invoke(prompt_templat)
S

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'O Ford Corcel II foi lançado no Brasil em 1976.'

In [15]:
print(S)

Por que o ovo foi ao médico?
Porque estava sentindo um pouco "cracked"!


### Modelos de Chat
*Aqui é a mesma lógica que foi visto antes (template, system_prompt, user_prompt, prompt_templa, nas linhas 12 a 17 ), mas agora usando classes langchain_core.messages.

*Esse formato será convertido em um prompt de string e passado para o LLM que fará o processamento.

In [20]:
msgs = [
    SystemMessage(content="Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content="Qual é o nome da pessoa que fundou Goiânia, no estado de Goiás no Brasil?")
]

chat_model = ChatHuggingFace(llm = llm)

In [7]:
#mode_templates = tokenizer.chat_template
# print(mode_templates)

In [21]:
print(chat_model._to_chat_prompt(msgs))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Você é um assistente e está respondendo perguntas gerais.<|eot_id|><|start_header_id|>user<|end_header_id|>

Qual é o nome da pessoa que fundou Goiânia, no estado de Goiás no Brasil?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [22]:
res = chat_model.invoke(msgs)
print(res.content)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A cidade de Goiânia, capital do estado de Goiás, no Brasil, foi fundada em 24 de outubro de 1946. A cidade foi criada a partir da fusão das cidades de Vila Boa e Inhomirim, que eram dois distritos do município de Goiás Velho.

O nome "Goiânia" é uma homenagem ao Imperador Pedro II do Brasil, que visitou a região em 1875 e se impressionou com a beleza do local. O imperador batizou o lugar de "Goiás", que significa "rio dos bugios", em alusão à tribo indígena que habitava na região.

Portanto, não há uma pessoa específica que tenha fundado Goiânia, pois a cidade foi criada por meio da fusão de dois distritos e recebeu seu nome em homenagem ao Imperador Pedro II.


### String PromptTemplates

In [23]:
prompt_templat = PromptTemplate.from_template("Escreva um poema sobre {topic}")
prompt_templat.invoke({"topic":"a cidade de Goiânia"})

StringPromptValue(text='Escreva um poema sobre a cidade de Goiânia')

In [26]:
PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

### ChatPromptTemplates

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "Explique-me em 1 parágrafo o conceito de {topic}")
])

prompt.invoke({"topic":"IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explique-me em 1 parágrafo o conceito de IA', additional_kwargs={}, response_metadata={})])

In [28]:
chain = prompt | llm  # LLM ustilizada é meta-llama/Meta-Llama-3-8B-Instruct, foi criado um chain para verificar o resultado junto ao modelo de LLM
chain.invoke({"topic":"IA"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


' (Inteligência Artificial).\nAssistente: A inteligência artificial (IA) é a capacidade de uma máquina ou programa de computador processar informações, aprender com elas e tomar decisões sem intervenção humana direta. Isso permite que as máquinas realizem tarefas complexas, como reconhecimento de padrões, resolução de problemas e tomada de decisões, de forma mais rápida e eficiente do que os seres humanos. A IA pode ser utilizada em uma ampla variedade de aplicações, desde sistemas de reconhecimento facial até robótica e inteligência assistiva. Em resumo, a IA é a capacidade de uma máquina de realizar tarefas que normalmente requeriam habilidades humanas, mas de forma mais rápida e eficiente.'

In [29]:
# Deixando dentro do prompt template para evitar que o modelo alucine

user_prompt = "Explique-me em 1 parágrafo o conceito de {topic}"

user_prompt_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(user_prompt)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", user_prompt_template)
])

chain = prompt | llm
chain.invoke({"topic": "IA"})

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'\n\nA Inteligência Artificial (IA) é a capacidade de uma máquina ou programa processar informações, aprender com elas e tomar decisões sem intervenção humana direta. Essa tecnologia permite que os computadores realizem tarefas que normalmente requeriam habilidades humanas, como a compreensão do linguagem natural, a análise de imagens e a tomada de decisões baseadas em dados. A IA é dividida em dois tipos principais: inteligência artificial limitada (IA limitada), que pode realizar apenas uma tarefa específica, e inteligência artificial geral (IA geral), que pode realizar qualquer tarefa que um ser humano seja capaz de fazer. Além disso, a IA também é classificada em subtipos, como aprendizado de máquina, visão computacional, reconhecimento de fala e linguagem natural. Em resumo, a IA é a capacidade dos computadores de processar informações, aprender e tomar decisões sem intervenção humana direta, permitindo que eles sejam cada vez mais eficientes e precisos na realização de tarefas co

In [30]:
print(user_prompt_template)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Explique-me em 1 parágrafo o conceito de {topic}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [31]:
print(prompt)

input_variables=['topic'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente e está respondendo perguntas gerais.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nExplique-me em 1 parágrafo o conceito de {topic}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), additional_kwargs={})]


<h4>Lembre-se que o template e tokens especiais pode mudar. Caso esteja definindo manualmente e não de modo automático usando o tokenizer então você deve sempre confira a descrição do modelo.<br><br>

Para avançarmos com o exemplos, é deixado preparado esse template</h4>

In [32]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo."

prompt = PromptTemplate.from_template(template.format(system_prompt = system_prompt, user_prompt = user_prompt))
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [33]:
prompt.invoke({"topic": "IA"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [34]:
# Adicionando mais uma variável 

user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}."
prompt = PromptTemplate.from_template(template.format(system_prompt=system_prompt, user_prompt=user_prompt))
prompt.invoke({"topic": "IA", "tamanho": "1 frase"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 frase.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [35]:
print(prompt)

input_variables=['tamanho', 'topic'] input_types={} partial_variables={} template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'
